In [ ]:
import tensorflow as tf
from keras import __version__
tf.keras.__version__ = __version__

import time
import random
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Embedding, Reshape

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

import gym

from taxi_env import TaxiPickupEnvSimplified, TaxiPickupEnvStandard, TaxiPickupEnvAdvanced

plt.style.use("ggplot")

# Part 3: Advanced environment (Deep Q-learning)

In this part, we will consider a more complex environment where pickup requests can appear anywhere on our 5x5 grid world. When there are requests in a map cell, we will represent that with a red counter showing the number of requests in that cell.

### Run Random Policy

As we have been doing before, let's start by having a look at how that environment looks like using a random policy:

In [ ]:
env = TaxiPickupEnvAdvanced()
env.reset()
# for _ in range(100):
#     try:
#         env.render()
#         state = env.step(env.action_space.sample()) # take a random action
#         time.sleep(1)
#         clear_output(wait=True)
#     except KeyboardInterrupt:
#         break
# env.close()

array([[[0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.]]])

In [ ]:
print("Action Space {}".format(env.action_space))

Action Space Discrete(5)


### Deep Q-learning (DQN)

As you probably realized already, when we start considering more interesting and more realistic problems, the dimensionality of the state space can quickly become unmanegeable for standard tabular Q-learning approaches. In the environment from Part 2, with just 3 possible pickup locations, the number of possible (discrete) states was already $5\times5\times2^3 = 200$. This is not a problem just because of memory requirements, but the learning task of fitting Q-table becomes much more complex (observations becomes sparser - curse of dimensionality)! Not only that, but what happens when the states are continuous rather than discrete? We definitely need a way of tackling these problems...

This is where function approximation comes in! In this tutorial, we will use neural networks as function approximators. This is also by far the most popular approach in the recent RL literature. To makes things easier, we will rely on 2 Python packages - keras and keras-rl - that abstract away many of the complexities involved in creating a neural network and using it for deep Q-learning.

**Optional: Implementing neural networks in Keras** (if you want, you may skip this part and treat the neural network as black-box function approximator)

Bulding a multi-layer neural network in Keras is fairly easy. We start by creating an object of the class "Sequential" (indicating that the neural network consists of sequence of layers):

```python
model = Sequential()
```

Now we can add layers to our neural network model. For examply, we can add a fully connected (dense) layer with 50 neurons and using a ReLU (rectified linear unit) activation function as follows:

```python
model.add(Dense(50, input_dim=30, activation='relu'))
```

A similar approach can be used for other types of layers such as Convolutional Layers. The following line adds a Convolutional layer with 20 filters of 3x3 convolutions:

```python
model.add(Conv2D(20, kernel_size=(3, 3), activation="relu"))
```
We can now keep adding more hidden layers, or add the final Dense layer. Note that since this is a regression problem (i.e. we want the neural network to output Q-value for the different possible actions given the state passed as input to it), the last layer (output layer) of network must necessarily have as many neurons/outputs as there are actions in our RL problem and it must use a linear activation:

```python
model.add(Dense(env.nA, activation='linear'))
```

**MDP formulation**:

Armed with the power of neural networks for function approximation, we can build a more complex representation of the environment state. In this case, we will represent the state using 4 5x5 matrices containing:

- Matrix 1: position of the taxi, one-hot encoded (i.e. with "1" in the place where the taxi is located, and zeros else where). For example:

``[[0,0,0,0,0],
 [0,0,0,0,0],
 [0,0,0,0,0],
 [0,0,0,1,0],
 [0,0,0,0,0]]``

- Matrix 2: number of pickup requests in each cell. For example:

``[[0,0,0,0,0],
 [0,0,0,1,0],
 [0,0,0,0,0],
 [2,0,0,0,4],
 [0,0,1,0,0]]``
 
- Matrix 3: information about the presence of walls to the east. In this case:

``[[0,1,0,0,0],
 [0,1,0,0,0],
 [0,0,0,0,0],
 [1,0,1,0,0],
 [1,0,1,0,0]]``
 
- Matrix 4: information about the presence of walls to the west. In this case:

``[[0,0,1,0,0],
 [0,0,1,0,0],
 [0,0,0,0,0],
 [0,1,0,1,0],
 [0,1,0,1,0]]``
 
We can stack these 4 matrices together to create a 5x5x4 tensor that represents the state of the environment, and feed it to the neural network as input. The idea is for the neural network to learn to approximate the Q-values (i.e. expected future rewards) for the different possible actions given the state that was passed as input.

Aside: we are not arguing that this is necessarily the best state representation for this problem. In fact, coming up with good state representation is a key challenge in deep RL and it can determine how fast your agent learns and how the policies that it learns can be. This state representation seems to perform acceptably well for this problem, so we will proceed with it.

In summary, the new MDP for this revised version of the problem can be formalized as:

**Actions:** north, south, east, west, pickup

**State:** 5x5x4 tensor representing the position of the taxi, the number of requests in each cell and the locations of the "walls"

**Reward:** +20 if taxi at a pickup location where there are requests and action is "pickup", else -1 (penalty for time elapsed); trying to pickup in a location different than the target also leads to penalty of -10.

The code below creates a neural network in Keras that performs well in this environment:

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Embedding, Reshape, Conv2D, MaxPooling2D, Input
from keras.optimizers import Adam

# First, we build a very simple neural network model in Keras
model = Sequential()
model.add(Input(shape=(1, env.num_rows, env.num_columns, 4)))
model.add(Reshape(target_shape=(env.num_rows, env.num_columns, 4)))
model.add(Conv2D(20, kernel_size=(3, 3), activation="relu"))
model.add(Flatten())
model.add(Dense(env.nA, activation='linear'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_2 (Reshape)         (None, 5, 5, 4)           0         
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 20)          740       
                                                                 
 flatten_2 (Flatten)         (None, 180)               0         
                                                                 
 dense_4 (Dense)             (None, 5)                 905       
                                                                 
Total params: 1645 (6.43 KB)
Trainable params: 1645 (6.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


Now that we have the neural network in place, it is time to use it as a function approximator for the Q-function of our RL agent. This can be easily done using the Keras-rl package in Python. The code below creates a deep Q-learning agent using $\epsilon$-greedy exploration:

In [ ]:
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy, LinearAnnealedPolicy
from rl.memory import SequentialMemory
from keras.src.saving import serialization_lib
serialization_lib.enable_unsafe_deserialization()
from tensorflow.keras.optimizers.legacy import Adam

# Then, define DQN agent in Keras-RL
memory = SequentialMemory(limit=200000, window_length=1)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), 
                              attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=100000)
dqn = DQNAgent(model=model, nb_actions=env.nA, memory=memory, policy=policy, 
               nb_steps_warmup=500, target_model_update=1e-2, enable_double_dqn=True, enable_dueling_network=True)
dqn.compile(optimizer=Adam(learning_rate=1e-3), metrics=['mae'])

In essence, we are doing the exact some thing as we did before: Q-learning with $\epsilon$-greedy exploration. The key difference is in the way that we approximate the Q-function: before we used a table, and now we are using a neural network. 

As you can probably guess from the code above, we are also using a few popular RL techniques that improve the stability and convergence of Q-learning algorithms:

- Experience replay: we add a ``memory`` that allows the RL agent to "re-live" past experience but accounting for the latest knowledge that it has about the Q-function.

- Double deep Q networks (``enable_double_dqn=True``)

- Dueling networks (``enable_dueling_network=True``)

These fall outside of the scope of this tutorial, but they are explained in detail in the aditional materials provided in the last slides.

We can now run our deep Q-learning algorithm (in this case for 400000 steps, where each episode has a maximum of 200 steps):

In [ ]:
dqn.fit(env, nb_steps=100000, visualize=False, verbose=1, nb_max_episode_steps=200, log_interval=10000)

Training for 100000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 59s 6ms/step - reward: -1.5229
50 episodes - episode_reward: -304.580 [-545.000, -98.000] - loss: 38.575 - mae: 63.957 - mean_q: 82.891 - mean_eps: 0.953

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 64s 6ms/step - reward: -1.1071
50 episodes - episode_reward: -221.420 [-386.000, -11.000] - loss: 89.134 - mae: 131.161 - mean_q: 167.712 - mean_eps: 0.865

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 66s 7ms/step - reward: -0.8044
50 episodes - episode_reward: -160.880 [-389.000, 109.000] - loss: 128.200 - mae: 164.491 - mean_q: 209.714 - mean_eps: 0.775

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 789s 79ms/step - reward: -0.6514
50 episodes - episode_reward: -130.280 [-341.000, 100.000] - loss: 143.402 - mae: 173.766 - mean_q: 221.247 - mean_eps: 0.685

Interval 5 (4000

Once the RL agent is learned, we can visualize the learned policy by exploiting the call-back functionality in Keras (don't worry if this code is a bit confusing for you as a first time Keras user; focus on the results that you obtained instead):

In [ ]:
from keras.callbacks import Callback

class Visualizer(Callback):
    def __init__(self, env):
        self.env = env
    
    def on_action_end(self, action, logs):
        """ Render environment at the end of each action """
        self.env.render(mode='human')
        time.sleep(1)
        clear_output(wait=True)

In [ ]:
try:
    dqn.test(env, nb_episodes=5, callbacks=[Visualizer(env)], nb_max_episode_steps=99, visualize=False, verbose=0)
except KeyboardInterrupt:
    pass

+---------+
| : | : : |
| :1| :1: |
| : : : : |
| | : | : |
| | : | : |
+---------+
T: 70; Total earnings: 120
Action: Pickup; Reward: 20


How are the results? Does the policy do what you would expect it to do? Or does it sometimes behaves strangely (i.e. not perfectly)? When I ran this code I managed to obtain a pretty good policy in a relatively short amount of time, but you can try doing a few tweaks to see if you improve. For example, try:

- Increasing the number of training steps ``nb_steps=400000``

- Increasing the maximum length of each episode ``nb_max_episode_steps=200``

- Increasing the complexity of the neural network (e.g. more layers or more neurons per layer)

- Changing the learning rate of the neural network optimizer ``Adam(lr=1e-3)``

- Other hyper-parameters of the RL algorithm (e.g. memory size, decay rate of $\epsilon$-greedy, etc.):

``memory = SequentialMemory(limit=200000, window_length=1)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), 
                              attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=100000)
dqn = DQNAgent(model=model, nb_actions=env.nA, memory=memory, policy=policy, 
               nb_steps_warmup=500, target_model_update=1e-2, enable_double_dqn=True, enable_dueling_network=True)``

That's it! We hope that you enjoyed this brief introduction to the world of reinforcement learning :-)